In [3]:
# 10/19/2021
# No need to run first section again. 
# Run import statements then proceed to Preprocess Data section where csv is loaded in.

# BINNING NEEDS WORK IN ORDER FOR MODELS TO RUN

In [3]:
# import dependencies
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
# from config import Password, Username, Host,DB
import pandas as pd

In [ ]:
# connect to database
engine = create_engine(f'postgresql://{Username}:{Password}@{Host}:5432/{DB}', echo=False)
connection = engine.connect()
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# fetch data from database
cursor = engine.execute('SELECT public."Movie"."movie_id", public."Movie"."movie_rating", public."Movie"."release_date", public."Movie"."runtime", public."Movie"."metascore", \
    public."Movie"."imdb_rating", public."Movie"."boxoffice", public."Movie"."budget", public."Movie"."buzz"  \
    FROM public."Movie"').fetchall()

In [ ]:
movies_df = pd.DataFrame(cursor, columns=["movie_id", "rating", "date", "runtime", "metascore", "imdb_rating", "boxoffice", "budget", "buzz"])
movies_df

In [ ]:
for index, row in movies_df.iterrows():
    # make sql call for people
    
    # make sql call for genres
    genre_query = engine.execute(f'SELECT public."Genre"."genre_name"\
        FROM public."Movie" \
        LEFT JOIN public."Movie_Genre_Junction" ON public."Movie"."movie_id" = public."Movie_Genre_Junction"."movie_id" \
        LEFT JOIN public."Genre" ON public."Movie_Genre_Junction"."genre_id" = public."Genre"."genre_id" \
        WHERE public."Movie"."movie_id" = {row["movie_id"]}').fetchall()
    for genre in genre_query:
        if genre[0] not in movies_df:
            movies_df[genre[0]] = 0
        movies_df.loc[movies_df["movie_id"] == row["movie_id"], [genre[0]]] = 1
    

In [ ]:
movies_df

In [ ]:
# save csv so we don't have to run the loop again
movies_df.to_csv("dummy_data_frame.csv", index=False)

# Preprocess Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [159]:
movies_df = pd.read_csv("/content/drive/MyDrive/final project/dummy_data_frame.csv")

In [160]:
movies_df

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz,Adventure,Comedy,Family,Animation,Action,Sci-Fi,Mystery,Thriller,Biography,Drama,Sport,Fantasy,Crime,Romance,Music,Musical,Horror,History,War,Documentary,News,Western,Short,Unnamed: 32
0,1,PG,2006-12-22,108,48,6.4,250863268,110000000,6,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,G,2006-06-09,117,73,7.1,244082982,70000000,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,PG-13,2006-05-26,104,58,6.7,234362462,210000000,23,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,PG-13,2006-05-19,149,46,6.6,217536138,125000000,7,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,PG-13,2006-06-28,154,72,6.0,200120000,232000000,15,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,6800,TV-MA,2021-02-12,88,44,5.5,892802,750000,4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1663,6803,R,2021-05-21,117,69,6.9,845464,1400000,24,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1664,6835,PG-13,2021-05-07,108,67,6.4,632542,15000000,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1665,6850,Not Rated,2021-06-18,101,65,6.9,546201,7000000,4,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [161]:
# separate out target: Box Office
# bin target data
!pip install jenkspy
import jenkspy
labels = ["0-10724389", "10724389-27367660", "27367660-46462469","46462469-67790117","67790117-92186262","92186262-120146040", 
            "120146040-153636354", "153636354-191719337", "191719337-228778661", "228778661-268488329","268488329-312433331", 
            "312433331-368065385", "368065385-459005868", "459005868-533720947", "533720947-760507625"]
labels_3 = ["65M", "202M", "760M"]
labels_4 = ["42M","121M","262M","760M"]
breaks = jenkspy.jenks_breaks(movies_df["boxoffice"], nb_class=4)
target = pd.cut(movies_df["boxoffice"] , bins=breaks, labels=labels_4, include_lowest=True)


In [162]:
breaks

[0.0, 41778863.0, 121463226.0, 262030663.0, 760507625.0]

In [163]:

movies_df.drop(columns=["boxoffice"], inplace=True)

In [164]:
# clean data
# date into month
movies_df["month"] = pd.DatetimeIndex(movies_df['date']).month


In [165]:
# make dummies for rating
dummy_df = pd.get_dummies(movies_df, columns=["rating"])

In [166]:
dummy_df.drop(columns=["movie_id", "date"], inplace=True)

In [168]:
# split data into test train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dummy_df, target, random_state=1)

In [169]:
# scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create Models

In [ ]:
# svc
from sklearn.svm import SVC
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

In [ ]:
# SVC Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

In [ ]:
# SVC predictions
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

In [170]:
# random forest
# need to bin data for classifications
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)

In [171]:
print(f'Training Score: {random_forest_model.score(X_train_scaled, y_train)}')
print(f'Testing Score: {random_forest_model.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.7074340527577938


In [ ]:
# random forest feature importances
import matplotlib.pyplot as plt

features = clf.feature_importances_
print(features)
plt.bar(x = range(len(features)), height=features)
plt.show()

In [172]:
# neural network
import tensorflow as tf
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid", input_dim=40))

nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 100)               4100      
_________________________________________________________________
dense_38 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_39 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 101       
Total params: 24,401
Trainable params: 24,401
Non-trainable params: 0
_________________________________________________________________


In [173]:
# neural net
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [174]:
# neural net
# Fit the model to the training data
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [175]:

fit_model = nn_model.fit(X_train_scaled, le.transform(y_train), epochs=200)

Epoch 1/200
40/40 [==============================] - 1s 3ms/step - loss: 0.7021 - accuracy: 0.4768
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 0.6779 - accuracy: 0.6328
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 0.6283 - accuracy: 0.6400
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 0.5027 - accuracy: 0.6256
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 0.2620 - accuracy: 0.6400
Epoch 6/200
40/40 [==============================] - 0s 3ms/step - loss: -0.0217 - accuracy: 0.6704
Epoch 7/200
40/40 [==============================] - 0s 3ms/step - loss: -0.2836 - accuracy: 0.6888
Epoch 8/200
40/40 [==============================] - 0s 3ms/step - loss: -0.4880 - accuracy: 0.6872
Epoch 9/200
40/40 [==============================] - 0s 2ms/step - loss: -0.6243 - accuracy: 0.6976
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: -0.8449 - accuracy: 0.6920
Epoc

In [176]:
# xg boost
from xgboost import XGBClassifier

# fit model no training data
xg_model = XGBClassifier()
xg_model.fit(X_train_scaled, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [177]:
# xg boost
# make predictions for test data
y_pred = xg_model.predict(X_test_scaled)
predictions = [value for value in y_pred]

from sklearn.metrics import accuracy_score
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.50%


# Output Models

In [180]:
import pickle
pickle.dump(xg_model, open("/content/drive/MyDrive/final project/xg_model.pkl", 'wb'))
# pickle.dump(nn_model, open("/content/drive/MyDrive/final project/nn_model.pkl", 'wb'))
pickle.dump(random_forest_model, open("/content/drive/MyDrive/final project/rf_model.pkl", 'wb'))